In [1]:
import json
import glob
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from sklearn.metrics import r2_score
# from sklearn.cluster import KMeans


from surprise import Dataset, Reader, KNNBasic, SVD, NMF, KNNWithMeans, SVDpp
from surprise.model_selection import train_test_split
from surprise import accuracy as sup_accuracy
from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.prediction_algorithms.matrix_factorization import SVDpp
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy

from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

In [ ]:
import torch
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',50)
pd.set_option('display.width',1000)
torch.manual_seed(2022)


In [15]:
cities_df = pd.read_csv("/Users/jianing/Documents/Study/Recsys/Hands-On-Recommender-System/data/travel_cities_info.csv")
user_ratings_df = pd.read_csv("/Users/jianing/Documents/Study/Recsys/Hands-On-Recommender-System/data/users_destinations.csv")

# data = data.iloc[:10000,]
print(cities_df.columns)
print(user_ratings_df.columns)




Index(['Destination', 'Keywords', 'Country'], dtype='object')
Index(['userId', 'username', 'gender', 'age', 'occupation', 'destination',
       'rating', 'interests'],
      dtype='object')


In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# 创建城市关键词向量
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
city_vectors = vectorizer.fit_transform(cities_df['Keywords'])

# 创建每个用户的偏好向量
user_preferences = []
for user_id in user_ratings_df['userId'].unique():
    user_data = user_ratings_df[user_ratings_df['userId'] == user_id]
    user_profile = user_data.merge(cities_df, left_on='destination', right_on='Destination', how='left')
    user_keywords_weights = user_profile.apply(lambda row: row['rating'] * city_vectors.getrow(cities_df.index[cities_df['Destination'] == row['destination']].tolist()[0]).toarray(), axis=1)
    user_profile_vector = sum(user_keywords_weights) / len(user_data)
    user_preferences.append((user_id, user_profile_vector))

# 根据用户偏好和未评分的城市推荐新的城市
recommendations = {}
for user_id, user_pref_vector in user_preferences:
    similarity_scores = cosine_similarity(user_pref_vector, city_vectors)[0]
    recommended_city_indices = similarity_scores.argsort()[::-1]
    recommended_cities = [cities_df.iloc[idx]['Destination'] for idx in recommended_city_indices if cities_df.iloc[idx]['Destination'] not in user_ratings_df[user_ratings_df['userId'] == user_id]['destination'].tolist()]
    recommendations[user_id] = recommended_cities[:3] 
print(recommendations)

/Users/jianing/miniconda3/envs/recsystest/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{1: ['Jakarta', 'Vancouver', 'Munich'], 2: ['Bangkok', 'Marrakech', 'Oslo'], 3: ['Colombo', 'Bangalore', 'Budapest'], 4: ['Kuala Lumpur', 'Quito', 'Mumbai'], 5: ['Amsterdam', 'Dar es Salaam', 'Reykjavik'], 6: ['Buenos Aires', 'Nairobi', 'Dubai'], 7: ['Warsaw', 'Zurich', 'Amsterdam'], 8: ['Santiago', 'Minsk', 'Melbourne'], 9: ['Oslo', 'Orlando', 'Mexico City'], 10: ['Kathmandu', 'Lima', 'St. Petersburg'], 11: ['Rio de Janeiro', 'Punta Cana', 'Florence'], 12: ['Oslo', 'Taipei', 'Phuket'], 13: ['Marrakech', 'Krakow', 'Bogota'], 14: ['Amsterdam', 'Yerevan', 'Barcelona'], 15: ['Cairo', 'Kathmandu', 'Vienna'], 16: ['Copenhagen', 'Vancouver', 'Accra'], 17: ['Panama City', 'St. Petersburg', 'Los Angeles'], 18: ['Kiev', 'San Jose (Costa Rica)', 'Doha'], 19: ['Belgrade', 'San Jose (Costa Rica)', 'Kathmandu'], 20: ['Quito', 'Bucharest', 'Yerevan'], 21: ['Kathmandu', 'Panama City', 'Phuket'], 22: ['Johannesburg', 'St. Petersburg', 'Shanghai'], 23: ['Bucharest', 'Geneva', 'Yerevan'], 24: ['Dubai', 

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 创建城市关键词向量
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
city_vectors = vectorizer.fit_transform(cities_df['Keywords']).toarray()

# 创建每个用户的偏好向量
user_preferences = []
for user_id in user_ratings_df['userId'].unique():
    user_data = user_ratings_df[user_ratings_df['userId'] == user_id]
    user_profile = user_data.merge(cities_df, left_on='destination', right_on='Destination', how='left')
    user_keywords_weights = user_profile.apply(lambda row: row['rating'] * city_vectors[cities_df.index[cities_df['Destination'] == row['destination']].tolist()[0]], axis=1)
    user_profile_vector = sum(user_keywords_weights) / len(user_data)
    user_preferences.append((user_id, user_profile_vector))

# 使用KNN进行推荐
recommendations = {}
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(city_vectors)

for user_id, user_pref_vector in user_preferences:
    distances, indices = knn.kneighbors([user_pref_vector], n_neighbors=3)
    recommended_cities = [cities_df.iloc[idx]['Destination'] for idx in indices[0] if cities_df.iloc[idx]['Destination'] not in user_ratings_df[user_ratings_df['userId'] == user_id]['destination'].tolist()]
    recommendations[user_id] = recommended_cities

print(recommendations)


In [22]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 创建城市关键词向量
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
city_vectors = vectorizer.fit_transform(cities_df['Keywords']).toarray()

# 创建用户兴趣向量
interests_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(', '))
user_interests = user_ratings_df.groupby('userId')['interests'].apply(lambda x: ', '.join(set(', '.join(x).split(', ')))).reset_index()
interests_vectors = interests_vectorizer.fit_transform(user_interests['interests']).toarray()

# 创建每个用户的综合偏好向量
user_preferences = []
for user_id in user_ratings_df['userId'].unique():
    user_data = user_ratings_df[user_ratings_df['userId'] == user_id]
    user_profile = user_data.merge(cities_df, left_on='destination', right_on='Destination', how='left')
    user_keywords_weights = user_profile.apply(lambda row: row['rating'] * city_vectors[cities_df.index[cities_df['Destination'] == row['destination']].tolist()[0]], axis=1)
    user_profile_vector = sum(user_keywords_weights) / len(user_data)
    
    # 获取用户兴趣向量
    if user_id in user_interests['userId'].values:
        user_interest_vector = interests_vectors[user_interests[user_interests['userId'] == user_id].index[0]]
    else:
        user_interest_vector = np.zeros_like(interests_vectors[0])
    
    # 合并用户偏好向量和兴趣向量
    combined_vector = np.concatenate((user_profile_vector, user_interest_vector))
    user_preferences.append((user_id, combined_vector))

# 使用KNN进行推荐
recommendations = {}
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(np.concatenate((city_vectors, np.zeros((len(city_vectors), interests_vectors.shape[1]))), axis=1))

for user_id, user_pref_vector in user_preferences:
    distances, indices = knn.kneighbors([user_pref_vector], n_neighbors=3)
    recommended_cities = [cities_df.iloc[idx]['Destination'] for idx in indices[0] if cities_df.iloc[idx]['Destination'] not in user_ratings_df[user_ratings_df['userId'] == user_id]['destination'].tolist()]
    recommendations[user_id] = recommended_cities[:3]  # 只推荐前3个城市

print(recommendations)

{1: [], 2: [], 3: ['Colombo', 'Bangalore'], 4: [], 5: [], 6: ['Buenos Aires'], 7: ['Warsaw'], 8: ['Santiago'], 9: [], 10: ['Kathmandu'], 11: ['Rio de Janeiro'], 12: ['Oslo'], 13: [], 14: [], 15: ['Cairo'], 16: ['Copenhagen'], 17: ['Panama City'], 18: ['Kiev'], 19: ['Belgrade'], 20: ['Quito'], 21: [], 22: [], 23: ['Bucharest'], 24: ['Dubai'], 25: ['Muscat'], 26: [], 27: [], 28: ['Lagos', 'Orlando'], 29: ['Toronto', 'Muscat'], 30: ['Barcelona', 'Bucharest'], 31: ['Islamabad'], 32: ['Montreal'], 33: ['Bucharest'], 34: [], 35: ['Muscat', 'Montreal'], 36: [], 37: [], 38: ['Lisbon', 'Oslo'], 39: ['Amsterdam'], 40: [], 41: ['Nairobi'], 42: ['Copenhagen'], 43: [], 44: [], 45: ['Los Angeles'], 46: ['Islamabad'], 47: ['Bangkok'], 48: [], 49: ['Hong Kong'], 50: ['Mumbai', 'Marrakech'], 51: ['Amsterdam'], 52: ['Las Vegas'], 53: ['Melbourne'], 54: [], 55: ['Phuket'], 56: [], 57: [], 58: [], 59: ['Mexico City'], 60: ['Panama City'], 61: ['Miami'], 62: [], 63: [], 64: ['Jerusalem'], 65: ['Yerevan'], 

/Users/jianing/miniconda3/envs/recsystest/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
cities_df['Keywords'] = cities_df['Keywords'].fillna('')
tfidf_matrix = tfidf.fit_transform(cities_df['Keywords'])

# 将TF-IDF矩阵转换为DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=cities_df.index.tolist())
print(tfidf_df.head())

city_encoder = LabelEncoder()
ity_encoder = LabelEncoder()
cities_df['cityId'] = city_encoder.fit_transform(cities_df['Destination'])
user_ratings_df['cityId'] = city_encoder.transform(user_ratings_df['destination'])


from sklearn.ensemble import RandomForestRegressor

# 假设ratings_df已经包含了用户评分数据
user_profiles = pd.merge(user_ratings_df, tfidf_df, left_on='cityId', right_index=True)

X = user_profiles.drop(['userId', 'cityId', 'rating'], axis=1)
X.columns = X.columns.astype(int)
y = user_profiles['rating']

# 训练模型
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

def recommend_cities(user_id, n_recommendations):
    # 假设所有城市特征已经是tfidf_df
    user_rated_cities = user_ratings_df[user_ratings_df['userId'] == user_id]['cityId']
    available_cities = cities_df[~cities_df['cityId'].isin(user_rated_cities)]

    # 预测评分
    predictions = model.predict(tfidf_df.loc[available_cities.index])
    recommended_idx = np.argsort(predictions)[-n_recommendations:]

    # 返回推荐城市
    return cities_df.iloc[recommended_idx]['Destination']

# 为用户推荐新城市
print(recommend_cities(1, 5))


# # 计算所有城市之间的相似度
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# # 城市名称到索引的映射
# city_to_index = {city: idx for idx, city in enumerate(cities_df['Destination'])}

# # 找到用户评分城市的索引
# user_indices = [city_to_index[city] for city in user_ratings['City']]

# # 计算这些城市与所有其他城市的平均相似度得分
# sim_scores = np.mean(cosine_sim[user_indices], axis=0)

# # 除去已评分的城市
# sim_scores[user_indices] = 0

# # 获取最高得分的城市索引
# recommended_city_indices = np.argsort(sim_scores)[-5:][::-1]

# 输出推荐的城市
# recommended_cities = cities_df['Destination'].iloc[recommended_city_indices]
# print(recommended_cities)

   0         1        2         3         4    5    6    7         8    9    \
0  0.0  0.000000  0.00000  0.199281  0.152250  0.0  0.0  0.0  0.000000  0.0   
1  0.0  0.000000  0.25358  0.000000  0.000000  0.0  0.0  0.0  0.000000  0.0   
2  0.0  0.000000  0.00000  0.000000  0.107741  0.0  0.0  0.0  0.000000  0.0   
3  0.0  0.000000  0.00000  0.000000  0.000000  0.0  0.0  0.0  0.140595  0.0   
4  0.0  0.269808  0.00000  0.000000  0.103217  0.0  0.0  0.0  0.000000  0.0   

   ...       110       111       112       113  114       115  116  117  \
0  ...  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.0   
1  ...  0.000000  0.000000  0.276498  0.000000  0.0  0.125348  0.0  0.0   
2  ...  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.0   
3  ...  0.000000  0.270727  0.000000  0.270727  0.0  0.000000  0.0  0.0   
4  ...  0.269808  0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.0   

        118       119  
0  0.000000  0.000000  
1  0.000000  0.000000  
2 

ValueError: invalid literal for int() with base 10: 'username'

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from torch_rechub.basic.features import SparseFeature, DenseFeature

# 对城市关键词进行TF-IDF处理
tfidf_vectorizer = TfidfVectorizer(max_features=100)
cities_df['keywords_tfidf'] = list(tfidf_vectorizer.fit_transform(cities_df['Keywords']).toarray())
keywords_features = tfidf_vectorizer.fit_transform(cities_df['Keywords']).toarray()
interests_features = tfidf_vectorizer.fit_transform(user_ratings_df['interests']).toarray()

# 对用户ID和城市名称进行编码
user_encoder = LabelEncoder()
user_ratings_df['userId'] = user_encoder.fit_transform(user_ratings_df['userId'])

city_encoder = LabelEncoder()
cities_df['cityId'] = city_encoder.fit_transform(cities_df['Destination'])
user_ratings_df['cityId'] = city_encoder.transform(user_ratings_df['destination'])

scaler = MinMaxScaler()
user_ratings_df['age'] = scaler.fit_transform(user_ratings_df[['age']])

user_features = [
    DenseFeature("interests_tfidf", dim=100),  # 假设interests的TF-IDF向量维度是100
    SparseFeature("user_id", vocab_size=user_encoder.classes_.size, embed_dim=16),
    DenseFeature("age", dim=1)
]

item_features = [
    SparseFeature("city_id", vocab_size=city_encoder.classes_.size, embed_dim=16),
    DenseFeature("keywords_tfidf", dim=100)  # 假设keywords的TF-IDF向量维度是100
]





TypeError: DenseFeature.__init__() got an unexpected keyword argument 'dim'

In [ ]:
from torch_rechub.basic.features import SparseFeature, SequenceFeature

# embed_dim 是指定 LabelEncoder 的维度，会通过训练来自动学习到合适的 Lookup table
user_features = [
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in user_cols
]
user_features += [
    SequenceFeature("hist_movie_id", vocab_size=feature_max_idx["movie_id"], embed_dim=16, pooling="mean", shared_with="movie_id") # mean pooling，会对历史观影的 embedding 做平均运算
]

item_features = [
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in item_cols
]

print(user_features[1].name)
print(user_features[1].get_embedding_layer())
print(user_features[1].get_embedding_layer()._parameters)


In [6]:
from torch_rechub.models.matching import DSSM
from torch_rechub.trainers import MatchTrainer
from torch_rechub.utils.data import MatchDataGenerator

model_config = {
    "user_tower": {"embedding_dim": 128, "hidden_units": (64, 32)},
    "item_tower": {"embedding_dim": 128, "hidden_units": (64, 32)},
}

model = DSSM(user_features, item_features, model_config)

data_gen = MatchDataGenerator(user_col='userId', item_col='cityId', label_col='rating', 
                              user_max_seq=10, batch_size=64)

train_loader, val_loader = data_gen.generate_dataloader(df_train=user_ratings_df)

trainer = MatchTrainer(model, train_loader, val_loader, 
                       optimizer="adam", 
                       epochs=10, 
                       lr=0.001,
                       verbose=1)

# 开始训练
trainer.train()



TypeError: DSSM.__init__() missing 3 required positional arguments: 'item_features', 'user_params', and 'item_params'

In [5]:
# 根据之前处理的数据拿到Dataloader
dg = MatchDataGenerator(x=x_train, y=y_train)
train_dl, test_dl, item_dl = dg.generate_dataloader(test_user, all_item, batch_size=256)

# 定义模型
model = DSSM(user_features, item_features, temperature=0.02,  # 在归一化之后的向量计算內积之后，乘一个固定的超参 r ，论文中命名为温度系数。归一化后如果不乘 temperature，模型无法收敛
             user_params={
                 "dims": [256, 128, 64],
                 "activation": 'prelu',  # important!!
             },
             item_params={
                 "dims": [256, 128, 64],
                 "activation": 'prelu',  # important!!
             })

# 模型训练器
trainer = MatchTrainer(model,
                       mode=0,  # 同上面的mode，需保持一致
                       optimizer_params={
                           "lr": 1e-4,
                           "weight_decay": 1e-6
                       },
                       n_epoch=10,
                       device='cpu',
                       model_path=save_dir)

# 开始训练
trainer.fit(train_dl)

NameError: name 'x_train' is not defined

In [ ]:
import collections
import numpy as np
import pandas as pd
from torch_rechub.utils.match import Annoy
from torch_rechub.basic.metric import topk_metrics

def match_evaluation(user_embedding, item_embedding, test_user, all_item, user_col='user_id', item_col='movie_id',
                     raw_id_maps="./raw_id_maps.npy", topk=10):
    print("evaluate embedding matching on test data")
    annoy = Annoy(n_trees=10)
    annoy.fit(item_embedding)

    #for each user of test dataset, get ann search topk result
    print("matching for topk")
    user_map, item_map = np.load(raw_id_maps, allow_pickle=True)
    match_res = collections.defaultdict(dict)  # user id -> predicted item ids
    for user_id, user_emb in zip(test_user[user_col], user_embedding):
        items_idx, items_scores = annoy.query(v=user_emb, n=topk)  #the index of topk match items
        match_res[user_map[user_id]] = np.vectorize(item_map.get)(all_item[item_col][items_idx])

    #get ground truth
    print("generate ground truth")

    data = pd.DataFrame({user_col: test_user[user_col], item_col: test_user[item_col]})
    data[user_col] = data[user_col].map(user_map)
    data[item_col] = data[item_col].map(item_map)
    user_pos_item = data.groupby(user_col).agg(list).reset_index()
    ground_truth = dict(zip(user_pos_item[user_col], user_pos_item[item_col]))  # user id -> ground truth

    print("compute topk metrics")
    out = topk_metrics(y_true=ground_truth, y_pred=match_res, topKs=[topk])
    return out

user_embedding = trainer.inference_embedding(model=model, mode="user", data_loader=test_dl, model_path=save_dir)
item_embedding = trainer.inference_embedding(model=model, mode="item", data_loader=item_dl, model_path=save_dir)

match_evaluation(user_embedding, item_embedding, test_user, all_item, topk=10, raw_id_maps=save_dir+"raw_id_maps.npy")
